# Корреляционный анализ

In [132]:
import pandas as pd
import numpy as np

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.
Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [133]:
data = pd.read_csv('water.txt', sep = '\t', header = 0, parse_dates = [2])

In [134]:
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [135]:
data.dtypes

location     object
town         object
mortality    object
hardness      int64
dtype: object

In [136]:
data1 = data[['mortality', 'hardness']].astype('float')

In [137]:
# pd.DataFrame.correlation принимает необязательный параметр method , указав, 
# какой алгоритм использовать. По умолчанию используется pearson .
round(data1['mortality'].corr(data1['hardness'], method='Wilcoxon stat'), 4)

-0.6548

В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [138]:
round(data1['mortality'].corr(data1['hardness'], method='spearman'), 4)

-0.6317

Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [139]:
data_n = data[data.location =='North']
data_n = data_n[['mortality', 'hardness']].astype('float')
pearson_n = round(data_n['mortality'].corr(data_n['hardness'], method='pearson'), 4)
data_s = data[data.location =='South']
data_s = data_s[['mortality', 'hardness']].astype('float')
pearson_s = round(data_s['mortality'].corr(data_s['hardness'], method='pearson'), 4)
print abs(pearson_n), abs(pearson_s)

0.3686 0.6022


Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.
Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки

In [140]:
a = 203.
b = 239.
c = 718.
d = 515.

In [141]:
from math import sqrt

In [142]:
MCC_x1x2 = (a*d - b*c)/sqrt((a+b)*(a+c)*(b+d)*(c+d)) 

In [143]:
round(MCC_x1x2, 3)

-0.109

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры.

In [144]:
from scipy.stats import chi2_contingency

In [145]:
bar_guesting = np.array([[a, b], [c, d]])

In [146]:
chi2_contingency(bar_guesting)

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[243.03402985, 198.96597015],
        [677.96597015, 555.03402985]]))

In [147]:
p_value = chi2_contingency(bar_guesting)[1]
print p_value

1.0558987006638725e-05


В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [148]:
w_share = round(float(a)/(a + c), 4) 
m_share = round(float(b)/(b + d), 4)
print m_share, m_share

0.317 0.317


In [149]:
from scipy import stats

In [150]:
def two_proprotions_confint(success_a, size_a, success_b, size_b, significance = 0.05):
#Parameters
#success_a, success_b : int
#    Number of successes in each group
#size_a, size_b : int
#    Size, or number of observations in each group
#significance : float, default 0.05
#Returns
#prop_diff : float
#    Difference between the two proportion
#confint : 1d ndarray
#    Confidence interval of the two proportion test
    prop_a = success_a / size_a
    prop_b = success_b / size_b
    var = prop_a * (1 - prop_a) / size_a + prop_b * (1 - prop_b) / size_b
    se = np.sqrt(var)
    # z critical value
    confidence = 1 - significance
    z = stats.norm(loc = 0, scale = 1).ppf(confidence + significance / 2)
    # standard formula for the confidence interval
    # point-estimtate +- z * standard-error
    prop_diff = prop_b - prop_a
    confint = prop_diff + np.array([-1, 1]) * z * se
    return prop_diff, confint

In [151]:
two_proprotions_confint(a, a+c, b, b+d)

(0.09656353231552606, array([0.05390523, 0.13922183]))

Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры.

In [152]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = np.sum(sample1)
    n2 = np.sum(sample2)
    
    p1 = sample1[0] / n1
    p2 = sample2[0] / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [153]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [154]:
proportions_diff_z_test(proportions_diff_z_stat_ind([a, c], [b, d]))

8.153453089576601e-06

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

                    Не доволен  Более или менее	Доволен
    Не очень счастлив   197	        111	          33

    Достаточно счастлив 382	        685	         331

         Очень счастлив 110	        342	         333

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [164]:
import pandas as pd
import scipy

df = pd.DataFrame({'a' : ['b', 'b','a', 'a' , 'a', 'a', 'a', 'a' , 'a', 'a','a', 'a', 'b', 'b', 'b', 'b', 'b', 'a','a','a', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'a', 'a'],
                   'b' : ['c','c', 'c','c', 'c', 'c','c', 'c', 'c','d' , 'd'  , 'd' , 'd' , 'd', 'd',  'c', 'd', 'c', 'd', 'c', 'd', 'c', 'd', 'c', 'd', 'c', 'd', 'c', 'd']})

In [165]:
pd.crosstab(df['a'], df['b'])

b,c,d
a,,
a,10,5
b,6,8


In [155]:
data = pd.DataFrame({'dissatisfied' : [197, 382, 110],
                     'more or less' : [111, 685, 342],
                     'satisfied' : [33, 331, 333]})

In [162]:
data.rename(index={0: 'not too happy', 1: 'happy enough', 2: 'very happy'},inplace=True)

In [163]:
data

,dissatisfied,more or less,satisfied
not too happy,197,111,33
happy enough,382,685,331
very happy,110,342,333


In [177]:
chi2, prob, df, expected = scipy.stats.chi2_contingency(np.array(data))

In [179]:
round(chi2, 4)

293.6831

На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры

In [182]:
prob

2.4964299580093467e-62

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

In [184]:
def cramers_stat(confusion_matrix):
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))

In [185]:
print('V Cramer stat value: %.4f' % cramers_stat(np.array(data)))

V Cramer stat value: 0.2412
